In [2]:
## 4.1 - BASIC OBJECT CLASSIFICATION USING PRE-TRAINED VGG16 MODEL

In [1]:
import numpy as np
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from keras.layers import Dense, Flatten, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator   # ✅ fixed import
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

# Load CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# ✅ Reduce dataset size (use only 10,000 training samples instead of 50,000)
x_train = x_train[:10000]
y_train = y_train[:10000]
x_test = x_test[:2000]
y_test = y_test[:2000]

# One-hot encode
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Build VGG16 transfer learning model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)   # smaller dense layer for speed
predictions = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Preprocess input
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(x_train)

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# ✅ Train for only 5 epochs
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=32),
    validation_data=(x_test, y_test),
    epochs=5,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


313/313 ━━━━━━━━━━━━━━━━━━━━ 154s 485ms/step - accuracy: 0.3594 - loss: 6.4520 - val_accuracy: 0.4825 - val_loss: 2.0801
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 150s 478ms/step - accuracy: 0.5118 - loss: 1.6766 - val_accuracy: 0.5415 - val_loss: 1.5670
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 149s 478ms/step - accuracy: 0.5832 - loss: 1.2240 - val_accuracy: 0.5450 - val_loss: 1.4455
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 150s 479ms/step - accuracy: 0.6209 - loss: 1.1077 - val_accuracy: 0.5795 - val_loss: 1.4017
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 151s 484ms/step - accuracy: 0.6527 - loss: 1.0096 - val_accuracy: 0.5660 - val_loss: 1.3660
Test Accuracy: 56.60%


In [ ]:
## 4.2 - IMAGENET CLASSIFICATION WITH DEEP RESIDUAL NETWORKS (RESNET)

In [4]:
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
import numpy as np

# Load pre-trained ResNet50 model
model = ResNet50(weights='imagenet')

# (Choose Option 1: upload or Option 2: download an image)
# Example: using Option 2
!wget -O elephant.jpg https://upload.wikimedia.org/wikipedia/commons/3/37/African_Bush_Elephant.jpg
img_path = "elephant.jpg"

# Preprocess image
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# Predict
predictions = model.predict(x)

# Decode top-3 predictions
decoded_preds = decode_predictions(predictions, top=3)[0]
print("Top 3 Predictions:")

for i, (imagenet_id, label, score) in enumerate(decoded_preds):
    print(f"{i+1}. {label} ({score*100:.2f}%)")

--2025-08-31 08:21:30--  https://upload.wikimedia.org/wikipedia/commons/3/37/African_Bush_Elephant.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.153.240, 2620:0:860:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.153.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4783815 (4.6M) [image/jpeg]
Saving to: ‘elephant.jpg’

elephant.jpg        100%[===================>]   4.56M  16.2MB/s    in 0.3s    

2025-08-31 08:21:30 (16.2 MB/s) - ‘elephant.jpg’ saved [4783815/4783815]

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Top 3 Predictions:
1. African_elephant (88.57%)
2. tusker (6.55%)
3. Indian_elephant (3.67%)
